# Text 생성 및 Prompt 예제

In [1]:
import os
import openai

from dotenv import load_dotenv
load_dotenv() # read local .env file

True

In [2]:
from openai import OpenAI
import google.generativeai as genai

client = OpenAI()

model_openai = "gpt-5-nano"
model_gemini = genai.GenerativeModel("gemini-2.5-flash")

model_openai, model_gemini

('gpt-5-nano',
 genai.GenerativeModel(
     model_name='models/gemini-2.5-flash',
     generation_config={},
     safety_settings={},
     tools=None,
     system_instruction=None,
     cached_content=None
 ))

## Responses API

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [3]:
system_prompt = "당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다."
user_input = "안갔어 시장에 그녀는."

# OpenAI  API 호출 예시
response = client.responses.create(
    model=model_openai,
    instructions=system_prompt,
    input=user_input
)

# 응답 결과 출력
print(response.output_text)

그녀는 시장에 가지 않았다.


In [4]:
# Gemini  API 호출 예시
prompt = f"""{system_prompt}

입력: {user_input}
출력:"""

response = model_gemini.generate_content(prompt)
# 응답 결과 출력
print(response.text)

그녀는 시장에 안 갔어.


## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

In [5]:
system_prompt = "구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다."
user_input = """
      최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다.
      회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다.
      네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데,
      이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
      이 데이터를 과일명, 색상, 맛으로 분석해 주세요.
      """

response = client.responses.create(
    model=model_openai,
    instructions=system_prompt,
    input=user_input
)

print(response.output_text)

다음은 과일명, 색상, 맛으로 정리한 CSV 형식 데이터입니다:
과일명,색상,맛
네오스키즐,보라색,사탕 맛
로헤클(loheckles),회색 빛이 도는 파란색,매우 시큼하고 레몬과 약간 비슷한
포유닛,밝은 녹색,풍미가 강함(단맛보다)
루프노바,네온 핑크색,네온 핑크 맛과 솜사탕 같은 맛
글로울(glowls),옅은 오렌지색,매우 강한 산미와 쓴맛(부식성)


In [6]:
prompt_csv = f"""{system_prompt}

입력:
{user_input}

CSV 형식으로 출력:"""

response = model_gemini.generate_content(prompt_csv)

print(response.text)

```csv
과일명,색상,맛
네오스키즐,보라색,사탕 맛
로헤클,회색 빛이 도는 파란색,매우 시큼한 맛 (레몬과 유사)
포유닛,밝은 녹색,풍미가 좋은 맛 (단맛보다 좋음)
루프노바,네온 핑크색,솜사탕 맛
글로울,옅은 오렌지색,매우 시고 쓴 맛 (부식성)
```


## Keyword 추출

In [7]:
system_prompt = "텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다."

text = """
"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다.
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다.
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다.
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다.
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다.
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."
"""

In [8]:
response = client.responses.create(
    model=model_openai,
    instructions=system_prompt,
    input=text
)

print(response.output_text)

- Black-on-Black ware
- Blackware
- New Mexico northern Pueblo pottery
- Pueblo potters/artists
- 20th century
- 21st century
- reduction firing
- surface burnishing
- burnishing
- fired slip
- slip
- incising and carving
- relief/raised surfaces
- Kha'po Owingeh
- P'ohwhóge Owingeh
- Pueblo families
- techniques passed down
- black background with black ceramic
- other Pueblo artists
- contemporary artists
- ancestral pottery


In [9]:
prompt_keywords = f"""{system_prompt}

텍스트:
{text}

키워드 목록:"""

response = model_gemini.generate_content(prompt_keywords)

print(response.text)

키워드 목록:

*   블랙 온 블랙 도자기 (Black-on-Black ware)
*   푸에블로 원주민
*   뉴멕시코 북부
*   도자기 전통
*   20세기 도자기
*   21세기 도자기
*   환원 소성 블랙웨어
*   선택적 버니싱
*   내화 슬립
*   조각 디자인
*   Kha'po Owingeh
*   P'ohwhóge Owingeh
*   현대 예술가
*   조상의 도자기


## 감성 분류기
- 한개의 text 감성 분석

In [10]:
system_prompt = "당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."
user_input = "나는 새로운 배트맨 영화가 좋습니다!"

response = client.responses.create(
    model=model_openai,
    instructions=system_prompt,
    input=user_input
)

print(response.output_text)

긍정적


In [11]:
prompt_sentiment = f"""{system_prompt}

입력: {user_input}
감정 분류:"""

response = model_gemini.generate_content(prompt_sentiment)

print(response.text)

긍정적


## 실습: 위의 Prompt 내용을 수정해 가며 api 실행